In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns= None

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from pandas_profiling import ProfileReport

In [4]:
data = pd.read_csv('train_Df64byy.csv')
data

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,50878,C4,845,Rented,Individual,22,22,No,X3,NaN,NaN,18,7704.0,0
50878,50879,C5,4188,Rented,Individual,27,27,No,X3,7.0,3.0,4,5408.0,0
50879,50880,C1,442,Rented,Individual,63,63,No,X2,14+,1.0,12,11374.0,0
50880,50881,C1,4,Owned,Joint,71,49,No,X2,2.0,2.0,16,28179.2,1


In [5]:
test = pd.read_csv('test_YCcRUnU.csv')
test.head(3)
test.shape

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0


(21805, 13)

In [6]:
df0 = pd.concat([data,test],axis=0,sort=True)
df0 = df0.set_index('ID')

In [7]:
df = df0.copy()

In [13]:
tgt = df[['Response']]
tgt.head(3)
tgt.value_counts()

,Response
ID,
1,0.0
2,0.0
3,1.0


Response
0.0         38673
1.0         12209
dtype: int64

In [16]:
df.fillna('Unknown',inplace=True)

In [17]:
df['Holding_Policy_Type'] = df['Holding_Policy_Type'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72687 entries, 1 to 72687
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Accomodation_Type        72687 non-null  object 
 1   City_Code                72687 non-null  object 
 2   Health Indicator         72687 non-null  object 
 3   Holding_Policy_Duration  72687 non-null  object 
 4   Holding_Policy_Type      72687 non-null  object 
 5   Is_Spouse                72687 non-null  object 
 6   Lower_Age                72687 non-null  int64  
 7   Reco_Insurance_Type      72687 non-null  object 
 8   Reco_Policy_Cat          72687 non-null  int64  
 9   Reco_Policy_Premium      72687 non-null  float64
 10  Region_Code              72687 non-null  int64  
 11  Response                 72687 non-null  object 
 12  Upper_Age                72687 non-null  int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 7.8+ MB


In [18]:
df.City_Code = df.City_Code.apply(lambda x: x.replace('C',''))
df['Health Indicator'] = df['Health Indicator'].apply(lambda x: x.replace('X',''))
df['Health Indicator'] = df['Health Indicator'].apply(lambda x: x.replace('Unknown','1'))
df['Holding_Policy_Type'] = df['Holding_Policy_Type'].apply(lambda x: x.replace('Unknown','3.0'))

In [36]:
age_cols = df[['Upper_Age','Lower_Age']]
age_cols = age_cols.mean(axis=1)

In [21]:
df.head(3)

,Accomodation_Type,City_Code,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Is_Spouse,Lower_Age,Reco_Insurance_Type,Reco_Policy_Cat,Reco_Policy_Premium,Region_Code,Response,Upper_Age
ID,,,,,,,,,,,,,
1,Rented,3,1,14+,3.0,No,36,Individual,22,11628.0,3213,0,36
2,Owned,5,2,Unknown,3.0,No,22,Joint,22,30510.0,1117,0,75
3,Owned,5,1,1.0,1.0,No,32,Individual,19,7450.0,3732,1,32


In [38]:
# df['Age'] = df['Upper_Age'].apply(lambda x:  if df['Reco_Insurance_Type']=='Individual' else 0)
# df.loc[df['Reco_Insurance_Type'] == 'Individual', 'Age'] = df['Lower_Age']
df.loc[df['Reco_Insurance_Type'] == 'Joint', 'Age'] = age_cols

In [40]:
df[['Holding_Policy_Duration']] = df[['Holding_Policy_Duration']].apply(lambda x: x.replace('Unknown','1.0'))
df[['Holding_Policy_Duration']] = df[['Holding_Policy_Duration']].apply(lambda x: x.replace('14+','15'))

In [42]:
df.drop(['Lower_Age','Upper_Age'],axis=1,inplace=True)

In [43]:
df_test = df.iloc[50882:,:]
df_train = df.iloc[:50882,:]

In [45]:
df_test.drop('Response',axis=1,inplace=True)

C:\Users\AHS\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [71]:
df_train.Response = df_train.Response.astype('int64')

C:\Users\AHS\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [106]:
# df_train = df_train.rename(columns={'Health Indicator' : 'Health_Indicator'})
df_train

,Accomodation_Type,City_Code,Health_Indicator,Holding_Policy_Duration,Holding_Policy_Type,Is_Spouse,Reco_Insurance_Type,Reco_Policy_Cat,Reco_Policy_Premium,Region_Code,Response,Age
ID,,,,,,,,,,,,
1,Rented,3,1,15,3.0,No,Individual,22,11628.0,3213,0,36.0
2,Owned,5,2,1.0,3.0,No,Joint,22,30510.0,1117,0,48.5
3,Owned,5,1,1.0,1.0,No,Individual,19,7450.0,3732,1,32.0
4,Owned,24,1,15,3.0,No,Joint,19,17780.0,4378,0,50.0
5,Rented,8,2,3.0,1.0,No,Individual,16,10404.0,2190,0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...
50878,Rented,4,3,1.0,3.0,No,Individual,18,7704.0,845,0,22.0
50879,Rented,5,3,7.0,3.0,No,Individual,4,5408.0,4188,0,27.0
50880,Rented,1,2,15,1.0,No,Individual,12,11374.0,442,0,63.0


In [107]:
train,val = train_test_split(df_train,test_size=0.2)

In [108]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Response')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [109]:
b = 5
train_ds = df_to_dataset(train,batch_size=b)
val_ds = df_to_dataset(val,shuffle=False,batch_size=b)
# test_ds = df_to_dataset(test,shuffle=False,batch_size=b)

In [119]:
def make_input_fn(dataframe, n_epochs=None, shuffle=True,batch_size=32):
  def input_fn():
    labels = dataframe.pop('h1n1_vaccine')
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
      dataset = dataset.shuffle(buffer_size=len(dataframe))
    # For training, cycle thru dataset as many times as need (n_epochs=None).    
    dataset = dataset.repeat(n_epochs)
    # In memory training doesn't use batching.
    dataset = dataset.batch(batch_size)
    return dataset
  return input_fn



train_input_fn  = make_input_fn(train)
val_input_fn  = make_input_fn(val,shuffle=False,n_epochs=1)

In [111]:
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [112]:
def demo(feat_col):
    f_layer = layers.DenseFeatures(feat_col)
    print(f_layer(example_batch).numpy())

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72687 entries, 1 to 72687
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Accomodation_Type        72687 non-null  object 
 1   City_Code                72687 non-null  object 
 2   Health Indicator         72687 non-null  object 
 3   Holding_Policy_Duration  72687 non-null  object 
 4   Holding_Policy_Type      72687 non-null  object 
 5   Is_Spouse                72687 non-null  object 
 6   Reco_Insurance_Type      72687 non-null  object 
 7   Reco_Policy_Cat          72687 non-null  int64  
 8   Reco_Policy_Premium      72687 non-null  float64
 9   Region_Code              72687 non-null  int64  
 10  Response                 72687 non-null  object 
 11  Age                      72687 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 9.7+ MB


In [83]:
NUMERIC_COLS = df.select_dtypes(include=['float64','int64']).columns.values
NUMERIC_COLS

array(['Reco_Policy_Cat', 'Reco_Policy_Premium', 'Region_Code', 'Age'],
      dtype=object)

In [85]:
numeric_cols = [feature_column.numeric_column(x) for x in NUMERIC_COLS]

In [93]:
vocab_cols = df.select_dtypes(exclude=['float64','int64']).columns.values
np.delete(vocab_cols,-1)

array(['Accomodation_Type', 'City_Code', 'Health Indicator',
       'Holding_Policy_Duration', 'Holding_Policy_Type', 'Is_Spouse',
       'Reco_Insurance_Type'], dtype=object)

In [94]:
feat_cols=numeric_cols
for x in list(vocab_cols):
    cat_col = feature_column.categorical_column_with_vocabulary_list(x, df[x].unique())
    indicator_col = feature_column.indicator_column(cat_col)
    feat_cols.append(indicator_col)

In [95]:
feature_layer = tf.keras.layers.DenseFeatures(feat_cols)

In [114]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test,shuffle=False,batch_size=batch_size)

In [115]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu', 
               kernel_regularizer=regularizers.l1_l2(l1=1e-2, l2=1e-2),
               bias_regularizer=regularizers.l2(1e-2),
               activity_regularizer=regularizers.l2(1e-2)
              ),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.2),
  layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])



In [116]:
for f,l in val_ds.take(1):    #take 1 batch
      print('Every feature:', f.keys())

Every feature: dict_keys(['Accomodation_Type', 'City_Code', 'Health_Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type', 'Is_Spouse', 'Reco_Insurance_Type', 'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Region_Code', 'Age'])


In [120]:
est = tf.estimator.BoostedTreesClassifier(feat_cols,
                                          n_batches_per_layer=batch_size)
est.train(train_input_fn, max_steps=100)
result = est.evaluate(val_input_fn)
result

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\AHS\\AppData\\Local\\Temp\\tmpfm09n44o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


KeyError: 'h1n1_vaccine'